In [ ]:
import torch 
import torchvision
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision.datasets import MNIST
import cv2
import os
import matplotlib.pyplot as plt
import torch.nn as nn
import os
from IPython.display import Image
from torchvision.utils import save_image


In [ ]:
mnist = MNIST(root = 'Trainig_data/',
             train = True,
             download = True, 
             transform = Compose([ToTensor(), Normalize(mean=(0.5), std = (0.5,))]))

In [ ]:
# Device Configuring for better performance
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:

import matplotlib.pyplot as plt
img, label = mnist[1]

In [ ]:
def denorm(x):
    out = (x+1) / 2
    return out.clamp(0, 1)

In [ ]:
%matplotlib inline
i = denorm(img)
plt.imshow(i[0], cmap = 'gray')
print('Label: ', label)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 100
data_loader = DataLoader(mnist, batch_size, shuffle = True)

batch size is some thing that is we are taking batches of 100 each so total baches will be 70000/100 i.e 700

In [ ]:
for img_batch, label_batch in data_loader:
    print('50th batch')
    print(img_batch[50].shape)
    plt.imshow(img_batch[50][0], cmap = 'gray')
    print( label_batch )
    break

In [ ]:
img_size = 784
hidden_size = 512

In [ ]:
import torch.nn as nn


D = nn.Sequential(
    nn.Linear(img_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

In [ ]:
D.to(device)

In [ ]:
latent_size = 64

In [ ]:
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, img_size),
    nn.Tanh())
  

In [ ]:
G.to(device)

In [ ]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(),lr = 0.0002)
g_optimizer = torch.optim.Adam(G.parameters(),lr = 0.0002)

In [ ]:
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()
    
    

In [ ]:
def train_discriminator(img):
    # Created the labels which are later used as input for the BCE loss
    real_labels = torch.ones(batch_size, 1).to(device)
    fake_labels = torch.zeros(batch_size, 1).to(device)
    
    #loss for real images
    outputs = D(img)
    d_loss_real = criterion(outputs, real_labels)
    real_score = outputs
    
    #loss for fake images
    z = torch.randn(batch_size, latent_size).to(device)
    fake_images = G(z)
    outputs = D(fake_images)
    d_loss_fake = criterion(outputs, fake_labels)
    fake_scores = outputs
    
    #combine losses
    d_loss = d_loss_real + d_loss_fake
    
    #Reset Gradients
    reset_grad()
    
    # Compute gradients
    d_loss.backward()
    
    #Adjust the parameters using backprop
    d_optimizer.step()
    
    return d_loss, real_score, fake_scores

In [ ]:
x = 15
for fake_images in range(x):
    z = torch.randn(batch_size, latent_size).to(device)
    fake_images = G(z)
    D(fake_images)
    print(denorm(fake_images))

In [ ]:
def train_generator():
    # Generate fake images and calculation loss
    z = torch.randn(batch_size, latent_size). to(device)
    fake_images = G(z)
    labels = torch.ones(batch_size, 1).to(device)
    g_loss = criterion(D(fake_images), labels)
    
    
    #Backprop and optimizer
    reset_grad()
    g_loss.backward()
    g_optimizer.step()
    return g_loss, fake_images
    

In [ ]:
import os 
sample_dir = 'samples/'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [ ]:
from IPython.display import Image
from torchvision.utils import save_image

for images, _ in data_loader:
    images = images.reshape(images.size(0), 1, 28, 28)
    save_image(denorm(images), 
                os.path.join(sample_dir, 'real_images.png'), 
                nrow = 10)
    
Image(os.path.join(sample_dir, 'real_images.png'))

In [ ]:
sample_vectors = torch.randn(batch_size, latent_size).to(device)

def save_fake_images(index):
    fake_images = G(sample_vectors)
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    fake_fname = 'fake_images-{0:0=4d}.png'.format(index)
    print('Saving', fake_fname)
    save_image(denorm(fake_images), os.path.join(sample_dir, fake_fname), nrow=10)
    
# Before training
save_fake_images(0)
Image(os.path.join(sample_dir, 'fake_images-0000.png'))

In [ ]:
%%time

num_epochs = 100
total_step = len(data_loader)
d_losses, g_losses, real_scores, fake_scores = [], [], [], []

for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        # Load a batch & transform to vectors
        images = images.reshape(batch_size, -1).to(device)
        
        # Train the discriminator and generator
        d_loss, real_score, fake_score = train_discriminator(images)
        g_loss, fake_images = train_generator()
        
        # Inspect the losses
        if (i+1) % 200 == 0:
            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())
            real_scores.append(real_score.mean().item())
            fake_scores.append(fake_score.mean().item())
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
        
    # Sample and save images
    save_fake_images(epoch+1)

In [ ]:
import cv2
import os
from IPython.display import FileLink

video_avi = 'GAN_Output.avi'

files = [os.path.join(sample_dir, f) for f in os.listdir(sample_dir) if 'fake_images' in f]
files.sort()

out = cv2.VideoWriter(video_avi,cv2.VideoWriter_fourcc(*'MP4V'), 8, (302,302))
[out.write(cv2.imread(fname)) for fname in files]
out.release()
FileLink('GAN_Output.avi')

In [ ]:
plt.plot(d_losses, '-')
plt.plot(g_losses, '-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Losses')

In [ ]:
plt.plot(real_scores, '-')
plt.plot(fake_scores, '-')
plt.xlabel('epoch')
plt.ylabel('score')
plt.legend(['Real Score', 'Fake score'])
plt.title('Scores');